In [145]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
import pandas as pd
import numpy as np
import etherscan
es = etherscan.Client('D8ATU83565DER66AM6MGXMB1XYYDNMY7TW')
from datetime import datetime

In [71]:
coin_cat = cg.get_coins_categories_list()
df_cat = pd.DataFrame(coin_cat, columns = ['category_id', 'name'])
mask = (df_cat['name'].str.contains('Ethereum')).any()
df_cat.loc[df_cat['name'] == 'Ethereum Ecosystem']

,category_id,name
44,ethereum-ecosystem,Ethereum Ecosystem


In [99]:
def get_data_on_page(page_number):
    return cg.get_coins_markets(vs_currency='USD', category = 'ethereum-ecosystem', page = page_number)

page_num = 1
eth_dapps = []
while(True):
    eth_dapps = eth_dapps + get_data_on_page(page_num)
    if len(get_data_on_page(page_num)) == 100:
        page_num +=1
    else:
        break

In [102]:
eth_dapps_df = pd.DataFrame(eth_dapps)
eth_dapps_df = eth_dapps_df[['id', 'symbol', 'name', 'last_updated']]

In [112]:
eth_test = eth_dapps_df['id']
eth_test.head()

0       ethereum
1         tether
2    binancecoin
3       usd-coin
4            okb
Name: id, dtype: object

In [127]:
contr_addr_list = []
gen_date_list = []
for idx, coin_id in enumerate(eth_test):
    sample_contract = cg.get_coin_by_id(id = coin_id)
    try:
        contr_addr_list.append(sample_contract['platforms']['ethereum'])
        gen_date_list.append(sample_contract['genesis_date'])
    except KeyError:
        contr_addr_list.append(np.nan)

    if idx == 2: break

gen_date_list

[None, '2017-07-08']

In [152]:
sample_coin_unix_date = cg.get_coin_market_chart_range_by_id(id='tether', vs_currency="eur", from_timestamp="631152000", to_timestamp="1735689600")

In [161]:
ts = int(sample_coin_unix_date['prices'][0][0])
try: 
    date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
except ValueError:
    ts /= 1000
    date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

date

'2015-02-25'

In [126]:
sample_contract.keys()

dict_keys(['id', 'symbol', 'name', 'asset_platform_id', 'platforms', 'detail_platforms', 'block_time_in_minutes', 'hashing_algorithm', 'categories', 'public_notice', 'additional_notices', 'localization', 'description', 'links', 'image', 'country_origin', 'genesis_date', 'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score', 'market_data', 'community_data', 'developer_data', 'public_interest_stats', 'status_updates', 'last_updated', 'tickers'])

In [128]:
sc_addresses = []
genesis_dates = []
length_of_eth_dapps = len(eth_dapps_df)
i = 0
for idx, coin_id in enumerate(eth_dapps_df['id']):
    coin = cg.get_coin_by_id(id = coin_id)
    try:
        sc_addresses.append(coin['platforms']['ethereum'])
        genesis_dates.append(coin['genesis_date'])
    except KeyError:
        sc_addresses.append(np.nan)
    
    if idx % 256 == 0:
        left =  length_of_eth_dapps - idx
        print(idx , " done and" , left , "left")

eth_dapps_df['contract_address'] = sc_addresses


0  done and 4062 left
256  done and 3806 left
512  done and 3550 left
768  done and 3294 left
1024  done and 3038 left
1280  done and 2782 left
1536  done and 2526 left
1792  done and 2270 left
2048  done and 2014 left
2304  done and 1758 left
2560  done and 1502 left
2816  done and 1246 left
3072  done and 990 left
3328  done and 734 left
3584  done and 478 left
3840  done and 222 left


In [164]:
# fetching dates into a list 
contr_addr_dates = []

for idx, coin_id in enumerate(eth_dapps_df['id']):
    try:
        sample_coin_unix_date = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency="eur", from_timestamp="631152000", to_timestamp="1735689600")
    except ValueError:
        contr_addr_dates.append(np.nan)
        continue

    ts = int(sample_coin_unix_date['prices'][0][0])
    try: 
        date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    except ValueError:
        ts /= 1000
        date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    except KeyError:
        date = np.nan

    if idx % 256 == 0:
        left =  length_of_eth_dapps - idx
        print(idx , " done and" , left , "left")

    contr_addr_dates.append(date)

eth_dapps_df['founding date'] = contr_addr_dates
eth_dapps_df.to_csv('final_contracts.csv')


0  done and 4062 left
256  done and 3806 left
512  done and 3550 left
768  done and 3294 left
1024  done and 3038 left
1280  done and 2782 left
1536  done and 2526 left


ValueError: {'error': 'coin not found'}

In [168]:
eth_dapps_df['founding date'] = contr_addr_dates

In [179]:
nan_id_rows = eth_dapps_df['id'][(eth_dapps_df['founding date'].isnull())]

In [198]:
nan_id_rows

1566                        alpha-coin
1567                            nftify
1568    bankless-defi-innovation-index
1569                             sator
1570                          bnpl-pay
                     ...              
4057                          ecredits
4058                  massive-protocol
4059                             freth
4060                     echelon-prime
4061                            lyfe-2
Name: id, Length: 2496, dtype: object

In [189]:
contr_addr_dates = contr_addr_dates[:1566]

In [201]:
for idx, coin_id in enumerate(nan_id_rows):
    try:
        sample_coin_unix_date = cg.get_coin_market_chart_range_by_id(id=coin_id, vs_currency="eur", from_timestamp="631152000", to_timestamp="1735689600")
    except ValueError:
        contr_addr_dates.append(np.nan)
        continue

    ts = int(sample_coin_unix_date['prices'][0][0])
    try: 
        date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    except ValueError:
        ts /= 1000
        date = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')
    except KeyError:
        date = np.nan

    if idx % 256 == 0:
        left =  length_of_eth_dapps - idx
        print(idx , " done and" , left , "left")

    contr_addr_dates.append(date)

eth_dapps_df['founding date_updated'] = contr_addr_dates
eth_dapps_df.to_csv('final_contracts.csv')


256  done and 3806 left
512  done and 3550 left
768  done and 3294 left
1024  done and 3038 left
1280  done and 2782 left
1536  done and 2526 left
1792  done and 2270 left
2048  done and 2014 left
2304  done and 1758 left


IndexError: list index out of range

In [202]:
len(contr_addr_dates)

4021

In [207]:
for i in range(len(eth_dapps_df) - len(contr_addr_dates)):
    contr_addr_dates.append(np.nan)

In [208]:
eth_dapps_df['founding date'] = contr_addr_dates

In [215]:
eth_dapps_df.to_csv('smart_contracts_updated.csv')

In [210]:
nan_id_rows = eth_dapps_df['id'][(eth_dapps_df['founding date'].isnull())]

In [214]:
nan_id_rows

1566                alpha-coin
2237                blocknotex
4021              1inch-yvault
4022                  ziticoin
4023                      1eco
4024                      zogi
4025              zombie-inu-2
4026                    zoodao
4027                 zoo-token
4028                 zudgezury
4029                 zum-token
4030                     zuplo
4031                  zurrency
4032                   12ships
4033              zuz-protocol
4034                     ystar
4035                   yvboost
4036                     yvdai
4037           zarp-stablecoin
4038                      1sol
4039                    zelwin
4040              20weth-80bal
4041           zunami-protocol
4042                xave-token
4043                      wusd
4044                     snkrz
4045                clever-cvx
4046    liquid-staked-ethereum
4047            smart-electrum
4048                      safe
4049            financie-token
4050                    exorde
4051    

In [133]:
len(genesis_dates)

4025

In [130]:

eth_dapps_df.to_csv('smart_contracts.csv')

In [32]:
print(sample_contract['genesis_date'])

None


In [6]:
len(contract_add_arr)

[nan,
 '0xdac17f958d2ee523a2206206994597c13d831ec7',
 nan,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
 '0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0',
 '0x75231f58b43240c9718dd58b4967c5114342a86c',
 '0x4fabb145d64652a948d72533023f6e7a623c7c53',
 '0xae7ab96520de3a18e5e111b5eaab095312d7fe84',
 '0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce',
 '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
 '0x6b175474e89094c44da98b954eedeac495271d0f',
 '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
 '0x514910771af9ca656af840dff83e8264ecf986ca',
 '0x582d872a1b094fc48f5de31d3b73f2d9be47def1',
 '0x2af5d2ad76741191d15dfe7bf6ac92d4bd912ca3',
 '0x5a98fcbea516cf06857215779fd812ca3bef1b32',
 '0x4d224452801aced8b2f0aebe155379bb5d594381',
 nan,
 '0x4a220e6096b25eadb88358cb44068a3248254675',
 '0xc944e90c64b2c07662a292be6244bdf05cda44a7',
 '0x3845badade8e6dff049820680d1f14bd3903a5d0',
 nan,
 '0x0f5d2fb29fb7d3cfee444a200298f468908cc942',
 '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9',
 '0xbb0e17ef65f82ab018d8edd776e8dd94

In [98]:
contract_addr_colmn = pd.DataFrame()
contract_Series = pd.Series(contract_add_arr)
contract_addr_colmn = pd.concat([contract_addr_colmn, contract_Series])
contract_addr_colmn.rename(columns = {0:'contract_address'}, inplace = True)
df_eth['contract_address'] = contract_addr_colmn['contract_address']

In [99]:
df_eth

,id,symbol,name,contract_address
0,ethereum,eth,Ethereum,NaN
1,tether,usdt,Tether,0xdac17f958d2ee523a2206206994597c13d831ec7
2,usd-coin,usdc,USD Coin,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
3,binance-usd,busd,Binance USD,0x4fabb145d64652a948d72533023f6e7a623c7c53
4,matic-network,matic,Polygon,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
...,...,...,...,...
95,woo-network,woo,WOO Network,0x4691937a7508860f876c9c0a2a617e7d9e945d4b
96,gains-farm,gfarm2,Gains Farm,0x831091da075665168e01898c6dac004a867f1e1b
97,nest,nest,Nest Protocol,0x04abeda201850ac0124161f037efd70c74ddc74c
98,constitutiondao,people,ConstitutionDAO,0x7a58c0be72be218b41c608b7fe7c5bb630736c71


In [116]:
df_stateDapps =  pd.read_csv('stateOfTheDapps.csv')
df_stateDapps['Contract Address (goerli)'].dropna() #, 'Contract Address (kovan)', 'Contract Address (mainnet)', 'Contract Address (rinkeby)', 'Contract Address (ropsten)']

# df_stateDapps = df_stateDapps[['dapps ', 'Headline', 'Site', 'GitHub', 'Platform', 'Status', 'Submitted', 'Last Update']]
# df_stateDapps = df_stateDapps.drop(df_stateDapps[df_stateDapps['Platform'] != 'Ethereum'].index)

385     0x35E08846892b323DE7Eeb0E20239b6ce18B7D7e7,\n0...
2955           0xE9DCe21085d14d8F8397Fbc76fD981756d511d99
3149           0x26e116efb304c73d0100ed8490a3abf9d0a652f7
3162           0xD3A09B674D8Fd6C544e9b44f8e16837ECCB11D02
3211           0xdcd25c8ad36970b5d6bbe066c7d240849fb08fd1
3262           0x2fD87516945cC2f1C52504f3BA3a24dF9918DC85
3299    0x09296686004F7A83DA07e0491720bbA1d85f013C,0x5...
3303           0xFDB25A324B66B9691E47FeE0d055B351461A7b74
3320           0x6011b6573fA152ded3d3188Ee6a90842BEa38b42
3403           0xcd40c4f617959338e47f453a666392bf10af5bc0
3406           0xb1e7fb9e9a71063ab552ddee87ea8c6eec7f5c7a
3460           0x60ca33852C7A0AF2E4c293eC14b74c3e623be681
3463           0x422a18Abd5F3FC957147fA752132eCe4571D8f7c
3504           0xD6242fF8Ac2ED87a26068D24843829405D34fd3f
3535           0x3C2D793b9a1Cf0cdc1e8a091Ae957747a5452727
3646           0x144662365A64C0f53296A65Bd33e2a7FDe4755A0
3800           0x55dc15f7961c7e37e63cb65438e818b6f95c4bd3
3921          

In [3]:
es.get_transactions_by_address(address='0xdac17f958d2ee523a2206206994597c13d831ec7')
# eth_price = es.get_eth_price()
# es.get_transactions_by_address(address='0xdac17f958d2ee523a2206206994597c13d831ec7')

[{'timestamp': 1511829681,
  'block_number': 4634748,
  'from': '0x36928500bc1dcd7af6a2b4008875cc336b927d57',
  'to': None,
  'input': '...',
  'hash': '0x2f1c5c2b44f771e942a8506148e256f94f1a464babc938ae0690c6e34cd79190',
  'value': 0,
  'gas': 3170794,
  'gas_price': 4000000000,
  'gas_used': 3170794,
  'nonce': 6,
  'confirmations': 11492330,
  'is_error': False,
  'tx_receipt_status': True,
  'transaction_index': 64,
  'cumulative_gas_used': 5915176,
  'block_hash': '0x1420761dbff0321c42a4bda163af2d7500d723b0139a72267c009d0eedfa9e6f'},
 {'timestamp': 1511831234,
  'block_number': 4634841,
  'from': '0x36928500bc1dcd7af6a2b4008875cc336b927d57',
  'to': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'input': '0xf2fde38b000000000000000000000000c6cde7c39eb2f0f0095f41570af89efc2c1ea828',
  'hash': '0x3796cbe75b68ffa2c98e737cb6e2034244c845764a76b02b174c298ea63caf10',
  'value': 0,
  'gas': 22680,
  'gas_price': 4000000000,
  'gas_used': 22680,
  'nonce': 7,
  'confirmations': 11492237,
 

In [111]:
# reqd_dates = []
df_stateDapps.rename(columns = {'dapps ':'dapps'}, inplace = True)
df_stateDapps['dapps'] = df_stateDapps['dapps'].str.lower()
df_eth['date submitted'] = np.where(df_stateDapps['dapps'] == df_eth['id'], 
                                df_stateDapps['Submitted'])
df_eth

ValueError: Can only compare identically-labeled Series objects